In [1]:
# প্রয়োজনীয় লাইব্রেরি ইন্সটলেশন (যদি প্রয়োজন হয়)
!pip install pandas scikit-learn --quiet

# প্রয়োজনীয় লাইব্রেরি ইমপোর্ট
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from google.colab import files

# 1. ডেটাসেট আপলোড করা
print("ফাইল আপলোড করুন:")
uploaded = files.upload()

# ফাইলের নাম সংগ্রহ করা
file_path = list(uploaded.keys())[0]

# 2. ডেটাসেট লোড করা
data = pd.read_csv(file_path, sep='\t')

# 3. ডেটাসেটের প্রথম কয়েকটি সারি দেখা
print("\nডেটাসেটের প্রথম কয়েকটি সারি:")
print(data.head())

# 4. ডেটা প্রস্তুতি
groups = data.iloc[0, 1:]  # গ্রুপের তথ্য
data = data.iloc[2:].set_index('id')  # শুধুমাত্র এক্সপ্রেশন ডেটা রেখে ইনডেক্স সেট করা
data = data.apply(pd.to_numeric, errors='coerce')  # সংখ্যায় রূপান্তর করা

# ইনপুট ফিচার (X) এবং টার্গেট লেবেল (y) প্রস্তুত
X = data.T  # ফিচার ডেটা (নমুনা অনুযায়ী ট্রান্সপোজ)
y = groups.values  # টার্গেট লেবেল (responder/nonresponder)

# 5. ডেটা ট্রেনিং ও টেস্ট সেটে ভাগ করা
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. মডেল তৈরি এবং ট্রেনিং
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 7. মডেল টেস্ট এবং একুরেসি গণনা
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 8. রেজাল্ট প্রিন্ট করা
print(f"\nমডেলের সঠিকতা (Accuracy): {accuracy:.2f}")


ফাইল আপলোড করুন:


Saving ExpressionDataPD1_filtered.txt to ExpressionDataPD1_filtered (1).txt

ডেটাসেটের প্রথম কয়েকটি সারি:
                   id   BS373_N1   BS373_N2      BS375_N1      BS375_N2  \
0              groups  responder  responder  nonresponder  nonresponder   
1  ENSG00000007314.10          8          0             0             0   
2   ENSG00000086696.9          5          0             0             0   
3   ENSG00000105668.6          0          0             4             0   
4  ENSG00000105695.13          0          1             0             0   

       BS379_N1      BS379_N2   BS381_N1   BS381_N2   BS385_N1  ...  \
0  nonresponder  nonresponder  responder  responder  responder  ...   
1             1             0          0          0          3  ...   
2             1             9          0          0          2  ...   
3             1             0          1          1          2  ...   
4             0             1          0          1          1  ...   

       BS391_N1

In [3]:
data = data.fillna(data.mean())  # মিসিং ভ্যালু গড় দিয়ে পূরণ


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)  # X ডেটা স্কেল করা


In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")


Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}


In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy: {scores.mean():.2f}")


Cross-Validation Accuracy: 0.75


In [11]:
print(f"NaN ভ্যালুর সংখ্যা (X): {pd.DataFrame(X).isna().sum().sum()}")


NaN ভ্যালুর সংখ্যা (X): 1


In [12]:
from sklearn.impute import SimpleImputer

# মিসিং ভ্যালু গড় দিয়ে পূরণ করা
imputer = SimpleImputer(strategy='mean')  # গড় দিয়ে NaN পূরণ
X_imputed = imputer.fit_transform(X)  # X ডেটাতে NaN পূরণ


In [13]:
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
import pandas as pd

# 1. NaN পূরণ করা (গড় দিয়ে)
imputer = SimpleImputer(strategy='mean')  # NaN পূরণের জন্য Imputer ব্যবহার
X_imputed = imputer.fit_transform(X)  # NaN পূরণ করা

# নিশ্চিত করা NaN আর নেই
print(f"NaN ভ্যালুর সংখ্যা (X_imputed): {pd.DataFrame(X_imputed).isna().sum().sum()}")  # 0 আশা করা হচ্ছে

# 2. SMOTE দিয়ে ব্যালেন্স করা
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_imputed, y)

# 3. ফিচার সিলেকশন
selector = SelectKBest(score_func=f_classif, k=50)
X_selected = selector.fit_transform(X_resampled, y_resampled)

# 4. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

# 5. মডেল তৈরি ও ট্রেনিং
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 6. মডেল টেস্ট এবং একুরেসি গণনা
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 7. ফলাফল প্রিন্ট করা
print(f"মডেলের সঠিকতা (Accuracy): {accuracy:.2f}")


NaN ভ্যালুর সংখ্যা (X_imputed): 0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


মডেলের সঠিকতা (Accuracy): 0.75
